In [185]:
import pandas as pd
import torch.nn as nn
import torch
import torch.optim as optim
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F
from tqdm.notebook import tqdm

In [186]:
#3 layer MLP with one hidden layer
class MultilayerPerceptron(nn.Module):
    #define network
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # input layer -> hidden layer
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(hidden_size, output_size)

    #define forward function
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        return out
    
    

In [298]:
def training(model, dataloader, func_loss, opt, num_epochs):
    model.train()
    
    for epoch in range(num_epochs):
        for i, batched_data in enumerate(dataloader): 
            inputs, targets = batched_data
#             print("inputs: " + str(inputs.shape))
#             print("target: " + str(targets.shape))
            
            output_prob = model(inputs) #pass images through model to get probability for each class (1 x 10 dim)
            loss = func_loss(output_prob.squeeze(), targets)
            
            #print(output_prob, targets)

            # Backward and optimize
            loss.backward() #performing back prop
            opt.step() #updating the weights
            opt.zero_grad() #reset gradients
            
        if epoch % 1 == 0:
            print('Epoch: ' + str(epoch) + ' Final loss is: ' + str(loss.item()))
            
    print('Epoch: ' + str(epoch) + ' Final loss is: ' + str(loss.item()))

In [309]:
df = pd.read_csv('../Data/PGH/DemandPrediction/ml_df.csv')
df = df.drop(columns=['Unnamed: 0', 'median_hh_income','mean_hh_income','in_labor_force', 'civ_labor_force', 
                      'employed','unemployed', 'perc_employed', 'perc_unemployed','own_alone', 
                      'carpool', 'public_transit', 'walked', 'other', 'wfh','mean_travel_time_to_work', 
                      'perc_public', 'perc_alone', 'perc_walk','perc_other', '2018_outflow', '2019_outflow', '2020_outflow'])

for row in range(len(df)):
    for col in df.columns:
        if df.loc[row][col] == '-' or pd.isna(df.loc[row][col]):
            df.at[row,col] = 0
            

X = torch.from_numpy(df.iloc[:, 0:3].to_numpy()).type(torch.float)
y = torch.from_numpy(df.iloc[:, 3].to_numpy()).type(torch.float)

# mean = torch.mean(X, axis=0)
# stdev = torch.std(X, axis=0)
# X = (X - mean)/stdev

dataset = TensorDataset(X, y)

train_ds, test_ds = random_split(dataset, [500, 228])
batch_size=1
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size, shuffle=True)

In [310]:
input_size = len(df.iloc[:,0:3].columns)
output_size = 1
input_size,output_size

(3, 1)

In [311]:
def evaluation(model, dataloader, func_loss):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        right, running_total = 0, 0

        for i, batched_data in enumerate(dataloader):

            #moving tensors to the GPU
            inputs, targets = batched_data #grab the batch of images as tensors and grab the batch of labels as tensors

            output_probs = model(inputs) #pass images through model to get probability for each class (1 x 10 dim)

            loss = func_loss(output_probs.squeeze(), targets)
            print("loss: " + str(loss))
            total_loss += loss
 
        print('Total Loss L1: ' + str(total_loss))

In [312]:
model = MultilayerPerceptron(input_size, 150, output_size)
model = nn.Sequential(nn.Linear(input_size, 5), nn.ELU(),
                     nn.Linear(5, 9), nn.ELU(),
                     nn.Linear(9, 9), nn.ELU(),
                     nn.Linear(9, 5), nn.ELU(),
                     nn.Linear(5, 1))
epochs = 500
learning_rate = 1e-4
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4, momentum=0.99)
training(model, train_loader, nn.L1Loss(), optimizer, epochs)

/Users/katelyncmorrison/opt/anaconda3/envs/CURF/lib/python3.7/site-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch: 0 Final loss is: 75.36445617675781
Epoch: 1 Final loss is: 39.07301330566406
Epoch: 2 Final loss is: 93.38905334472656
Epoch: 3 Final loss is: 90.29779052734375
Epoch: 4 Final loss is: 75.46408081054688
Epoch: 5 Final loss is: 166.05946350097656
Epoch: 6 Final loss is: 132.12452697753906
Epoch: 7 Final loss is: 132.12359619140625
Epoch: 8 Final loss is: 98.99897766113281
Epoch: 9 Final loss is: 66.23530578613281
Epoch: 10 Final loss is: 55.36125183105469
Epoch: 11 Final loss is: 294.28607177734375
Epoch: 12 Final loss is: 4.2451019287109375
Epoch: 13 Final loss is: 221.8878173828125
Epoch: 14 Final loss is: 9.013427734375
Epoch: 15 Final loss is: 190.3286590576172
Epoch: 16 Final loss is: 88.96755981445312
Epoch: 17 Final loss is: 2068.10791015625
Epoch: 18 Final loss is: 38.760711669921875
Epoch: 19 Final loss is: 75.56967163085938
Epoch: 20 Final loss is: 0.20469045639038086
Epoch: 21 Final loss is: 4.0063323974609375
Epoch: 22 Final loss is: 168.40122985839844
Epoch: 23 Final

In [260]:
evaluation(model, test_loader, nn.L1Loss())

loss: tensor(138.0162)
loss: tensor(121.0035)
loss: tensor(592.4202)
loss: tensor(470.4689)
loss: tensor(203.6801)
loss: tensor(110.0096)
loss: tensor(123.6990)
loss: tensor(173.9393)
loss: tensor(0.0999)
loss: tensor(0.0999)
loss: tensor(195.3358)
loss: tensor(147.4194)
loss: tensor(178.4049)
loss: tensor(146.3742)
loss: tensor(565.1467)
loss: tensor(146.5614)
loss: tensor(23.6305)
loss: tensor(394.2853)
loss: tensor(366.4063)
loss: tensor(494.4037)
loss: tensor(23.6305)
loss: tensor(39.8992)
loss: tensor(189.4347)
loss: tensor(173.9393)
loss: tensor(237.5204)
loss: tensor(420.1152)
loss: tensor(187.2990)
loss: tensor(205.6336)
loss: tensor(50.6057)
loss: tensor(118.6045)
loss: tensor(497.5378)
loss: tensor(19.3401)
loss: tensor(0.0999)
loss: tensor(183.5716)
loss: tensor(329.0051)
loss: tensor(305.0688)
loss: tensor(143.9107)
loss: tensor(184.3481)
loss: tensor(1122.0887)
loss: tensor(117.6321)
loss: tensor(1122.0887)
loss: tensor(23.6305)
loss: tensor(237.5204)
loss: tensor(141.7822